# First trajectory-optimization problems
This notebook drives you through simple trajectory optimization problems, considering only "kinematic" version i.e only motion variables (no forces or torques yet). 
We will first look at a problem where the trajectory is described by a sequence of configurations $[q_0 ... q_T]$, with the only constraints is to have them sufficiently closed to each other.
The we will look at a real integration, where each timestep is described by a configuration $q_t$, velocity $v_t$ and acceleration $a_t$ (all in the configuration space. 
This will allow us to formulate a proper contact constraint, even if only partially described from the motion point of view. 
The next notebook will drive us to complete version of the trajectory optimization problem, including forces, torques and complete contacts.



In [3]:
import magic_donotload

NB: as for all the tutorials, a magic command %do_not_load is introduced to hide the solutions to some questions. Change it for %load if you want to see (and execute) the solution.


## Set up
We will use the following tools:
- the ur10 model (loaded by example-robot-data)
- pinocchio.casadi for writing the problem and computing its derivatives
- the IpOpt solver wrapped in casadi
- the meshcat viewer

In [3]:
# %load tp2/generated/trajopt_kine_imports
import time
import unittest
import numpy as np
import pinocchio as pin
import casadi
from pinocchio import casadi as cpin
import example_robot_data as robex
from scipy.optimize import fmin_bfgs
from numpy.linalg import norm

from utils.meshcat_viewer_wrapper import MeshcatVisualizer


The notebook is written first for a UR10 robot model configured in fixed basis, then with a Talos pair of legs configured in floating mode. Replace the robot by your own URDF...

In [4]:
robot = robex.load('ur10')

The robot will start from an arbitrary initial configuration (make the solver life simpler by avoiding the singular 0 configuration) and aim at an object located a placement $M_{target} \in SE(3)$

In [9]:
# %load tp2/generated/trajopt_kine_configurations
Mtarget = pin.SE3(pin.utils.rotate('y', 3), np.array([-0.5, 0.1, 0.2]))  # x,y,z
q0 = np.array([0, -3.14 / 2, 0, 0, 0, 0])
endEffectorFrameName = 'tool0'


Open a viewer, display the robot and markers to visualize

In [11]:
# %load tp2/generated/trajopt_kine_viewer
# Open the viewer
viz = MeshcatVisualizer(robot)
viz.display(robot.q0)
# Add a vizualization for the target
boxID = "world/box"
viz.addBox(boxID, [.05, .1, .2], [1., .2, .2, .5])
# Add a vizualisation for the tip of the arm.
tipID = "world/blue"
viz.addBox(tipID, [.08] * 3, [.2, .2, 1., .5])
def displayScene(q,dt=1e-1):
    '''
    Given the robot configuration, display:
    - the robot
    - a box representing endEffector_ID
    - a box representing Mtarget
    '''
    pin.framesForwardKinematics(model,data,q)
    M = data.oMf[endEffector_ID]
    viz.applyConfiguration(boxID, Mtarget)
    viz.applyConfiguration(tipID, M)
    viz.display(q)
    time.sleep(dt)


Wrapper tries to connect to server <tcp://127.0.0.1:6000>
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [12]:
viz.viewer.jupyter_cell()

We are mostly interested by the robot model. We also need to access some particular frames.

In [14]:
# %load tp2/generated/trajopt_kine_modeldata
robot.q0 = q0

# The pinocchio model is what we are really interested by.
model = robot.model
data = model.createData()
endEffector_ID = model.getFrameId(endEffectorFrameName)


Finally, we will need the same helpers for getting around the Casadi SX/MX issue.

In [16]:
# %load tp2/generated/trajopt_kine_casadi
# --- Casadi helpers
cmodel = cpin.Model(model)
cdata = cmodel.createData()

cq = casadi.SX.sym("x",model.nq,1)
cpin.framesForwardKinematics(cmodel,cdata,cq)
pos_tool = casadi.Function('ptool', [cq], [ cdata.oMf[endEffector_ID].translation ])
error_tool = casadi.Function('etool', [cq],
                             [ cpin.log6(cdata.oMf[endEffector_ID].inverse() * cpin.SE3(Mtarget)).vector ])
